In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn
# import seaborn as sns; sns.set()
# from lifelines import KaplanMeierFitter

# from lifelines.utils import median_survival_times

# from lifelines.statistics import pairwise_logrank_test

from sklearn.preprocessing import OneHotEncoder

In [17]:
forecast_starting_data = pd.read_csv("forecast_starting_data.csv")
# macro_df = pd.read_csv("macro_data.csv")
macro_data = pd.read_csv("macro_data.csv",skiprows = range(1,8))
macro_data["Mnemonic"] = pd.to_datetime(macro_data["Mnemonic"])
macro_data["Mnemonic"] = macro_data["Mnemonic"].apply(lambda x: int(x.strftime('%Y%m')))
macro_data.head()

Mnemonic  M_FHOFHOPIPOQ.IUSA  M_FYLE.IUSA  M_FRT.IUSA  M_FYPSAVQ.IUSA  \
0    200001          136.384471     5715.426    2921.232             5.0   
1    200002          136.175916     5754.412    2965.596             4.5   
2    200003          137.238976     5777.722    2997.900             4.3   
3    200004          138.189028     5787.542    2949.972             4.8   
4    200005          138.787406     5772.214    2954.412             4.8   

   M_FC.IUSA  FYPCPICA_FEDB.IUSA   M_FGDP.IUSA  M_FYHHAVGQ.IUSA  \
0   6542.870         29747.16297   9914.297967      79259.05028   
1   6625.259         29943.16651   9989.058091      79784.01732   
2   6686.521         30136.36473  10103.183660      80309.11267   
3   6679.067         30326.88986  10177.387220      80643.86257   
4   6709.732         30508.25037  10265.088090      80969.88063   

   M_FYFMMEDQ.IUSA  ...  M_FBCFCOFCCQ.IUSA  M_FETP.IUSA  M_FLTV.IUSA  \
0      51147.36716  ...           4.562116      110.434    78.900002   
1      51287.26889  ...           4.531455      110.525    78.900002   
2      51422.36774  ...           4.376417      110.863    78.400002   
3      51553.75004  ...           4.169691      111.086    78.300003   
4      51677.16706  ...           4.031345      110.958    78.400002   

    M_FC$.IUSA  M_FEIAMGTTQ.IUSA  M_FXCPMGASUQ.IUSA  M_FRBBBQ.IUSA  \
0  8450.176371           1.32860         131.753937       8.074702   
1  8531.737106           1.41475         140.055069       8.189753   
2  8580.387368           1.55575         153.445095       8.338543   
3  8569.820795           1.50575         150.435879       8.469916   
4  8610.546128           1.52640         150.625958       8.509325   

   M_FHOWNRQ.IUSA  M_FIFR_FEDS.IUSA  M_FIFR$_FEDS.IUSA  
0       67.062138        481.331342         654.113753  
1       67.093048        484.649042         654.796053  
2       67.144365        486.748616         654.968193  
3       67.215255        487.541377         654.183206  
4       67.298718        486.974022         651.820675  

[5 rows x 97 columns]

In [7]:
for i in one_hot:
    print(forecast_starting_data[i].unique())

['T' 'P' 'C' 'R' 'D' 'I']
['I 0-6' 'A' 'I 13-24' 'I 25+' 'I 7-12' 'W' 'Recovery']
['761+' '621-640' '641-660' '701-720' '721-740' '681-700' '661-680'
 '741-760' '<= 560' '581-600' '601-620' '561-580' 'Exception' 'Missing']
['B' 'A' 'C']


In [3]:
def delta_month(x):
    if x > 12:
        month = x - 100 + 12
    else: 
        month = x
    return(month)

In [18]:
def prepossessing(df):
    df["month"] = (df["mth_code"] - df["snapshot"]).apply(delta_month)
    df.drop(["charge_off_reason_code", "variable_rate_margin", "writeoff_date"], axis=1, inplace=True)
    df["net_finance_charge"].fillna(0, inplace=True)
    one_hot = ["net_payment_behaviour_tripd", "account_status_code", "bank_fico_buckets_20", "industry"]
    encoder = OneHotEncoder()
    one_code_df = pd.DataFrame(encoder.fit_transform(df[one_hot]).toarray())
    one_code_df.columns = encoder.get_feature_names_out(one_hot)
    df = df.join(one_code_df)
    df.drop(one_hot, axis=1, inplace=True)
    df["account_status_code_MonthEnd"] = 0
    df["account_status_code_Clchange"] = 0
    df = pd.merge(df, macro_data, left_on="mth_code", right_on="Mnemonic", how="left")
    df.drop(["mth_code", "snapshot", "Mnemonic"], axis=1, inplace=True)
    selected_features = ['charge_off','month','aged_writeoff_amt', 'active', 'principal_amt_chrg_off','fee_reversals', 'total_writeoff_amt',
       'M_FLBT.IUSA', 'M_FCBC.IUSA', 'bank_fico_buckets_20_561-580','bank_fico_buckets_20_581-600', 'bank_fico_buckets_20_601-620',
       'bank_fico_buckets_20_621-640', 'bank_fico_buckets_20_641-660',
    'bank_fico_buckets_20_661-680', 'bank_fico_buckets_20_681-700',
       'bank_fico_buckets_20_701-720', 'bank_fico_buckets_20_721-740',
    'bank_fico_buckets_20_741-760', 'bank_fico_buckets_20_761+',
       'bank_fico_buckets_20_<= 560', 'bank_fico_buckets_20_Exception',
    'bank_fico_buckets_20_Missing',
       'M_FHST.IUSA', 'M_FIFR_FEDS.IUSA', 'M_FIFR$_FEDS.IUSA',"charge_off_aged", 'fc_reversals', 'M_FIP.IUSA',"writeoff_type_bko","writeoff_type_fraud_kiting","writeoff_type_fraud_synthetic","writeoff_type_deceased","writeoff_type_other","writeoff_type_aged","writeoff_type_settlement","writeoff_type_fraud_other","writeoff_type_repo","writeoff_type_null"]
    
    df = df.loc[:,selected_features]
    return df
    
    
    

In [19]:
selected_forecast_starting_data = prepossessing(forecast_starting_data)
selected_forecast_starting_data

charge_off  month  aged_writeoff_amt  active  principal_amt_chrg_off  \
0               0      0                0.0       1                     0.0   
1               0      0                0.0       1                     0.0   
2               0      0                0.0       1                     0.0   
3               0      0                0.0       1                     0.0   
4               0      0                0.0       1                     0.0   
...           ...    ...                ...     ...                     ...   
19861           0      0                0.0       1                     0.0   
19862           0      0                0.0       1                     0.0   
19863           0      0                0.0       1                     0.0   
19864           0      0                0.0       1                     0.0   
19865           0      0                0.0       1                     0.0   

       fee_reversals  total_writeoff_amt  M_FLBT.IUSA  M_FCBC.IUSA  \
0                0.0                 0.0         63.4        130.4   
1                0.0                 0.0         63.4        130.4   
2                0.0                 0.0         63.4        130.4   
3                0.0                 0.0         63.4        130.4   
4                0.0                 0.0         63.4        130.4   
...              ...                 ...          ...          ...   
19861            0.0                 0.0         63.4        130.4   
19862            0.0                 0.0         63.4        130.4   
19863            0.0                 0.0         63.4        130.4   
19864            0.0                 0.0         63.4        130.4   
19865            0.0                 0.0         63.4        130.4   

       bank_fico_buckets_20_561-580  ...  writeoff_type_bko  \
0                               0.0  ...                  0   
1                               0.0  ...                  0   
2                               0.0  ...                  0   
3                               0.0  ...                  0   
4                               0.0  ...                  0   
...                             ...  ...                ...   
19861                           0.0  ...                  0   
19862                           0.0  ...                  0   
19863                           0.0  ...                  0   
19864                           0.0  ...                  0   
19865                           0.0  ...                  0   

       writeoff_type_fraud_kiting  writeoff_type_fraud_synthetic  \
0                               0                              0   
1                               0                              0   
2                               0                              0   
3                               0                              0   
4                               0                              0   
...                           ...                            ...   
19861                           0                              0   
19862                           0                              0   
19863                           0                              0   
19864                           0                              0   
19865                           0                              0   

       writeoff_type_deceased  writeoff_type_other  writeoff_type_aged  \
0                           0                    0                   0   
1                           0                    0                   0   
2                           0                    0                   0   
3                           0                    0                   0   
4                           0                    0                   0   
...                       ...                  ...                 ...   
19861                       0                    0                   0   
19862                       0                    0 

In [20]:
selected_forecast_starting_data.drop(["charge_off","month"], axis = 1,inplace = True)
selected_forecast_starting_data.to_csv("selected_forecast_starting_data.csv", index=False)

In [4]:
print(forecast_starting_data.isna().sum().to_string())

financial_active                     0
net_payment_behaviour_tripd          0
promotion_flag                       0
variable_rate_index                  0
account_status_code                  0
active_12_mths                       0
bank_fico_buckets_20                 0
charge_off_reason_code           19866
mob                                  0
open_closed_flag                     0
ever_delinquent_flg                  0
nbr_mths_due                         0
variable_rate_margin             15089
stmt_balance                         0
prev_balance                         0
net_sales                            0
net_payments                         0
purchase_active                      0
credit_limit_amt                     0
credit_limit_pa                      0
closed                               0
active                               0
charge_off                           0
charge_off_aged                      0
charge_off_bk                        0
principal_amt            

In [21]:
sorted(selected_forecast_starting_data.columns)

['M_FCBC.IUSA',
 'M_FHST.IUSA',
 'M_FIFR$_FEDS.IUSA',
 'M_FIFR_FEDS.IUSA',
 'M_FIP.IUSA',
 'M_FLBT.IUSA',
 'active',
 'aged_writeoff_amt',
 'bank_fico_buckets_20_561-580',
 'bank_fico_buckets_20_581-600',
 'bank_fico_buckets_20_601-620',
 'bank_fico_buckets_20_621-640',
 'bank_fico_buckets_20_641-660',
 'bank_fico_buckets_20_661-680',
 'bank_fico_buckets_20_681-700',
 'bank_fico_buckets_20_701-720',
 'bank_fico_buckets_20_721-740',
 'bank_fico_buckets_20_741-760',
 'bank_fico_buckets_20_761+',
 'bank_fico_buckets_20_<= 560',
 'bank_fico_buckets_20_Exception',
 'bank_fico_buckets_20_Missing',
 'charge_off_aged',
 'fc_reversals',
 'fee_reversals',
 'principal_amt_chrg_off',
 'total_writeoff_amt',
 'writeoff_type_aged',
 'writeoff_type_bko',
 'writeoff_type_deceased',
 'writeoff_type_fraud_kiting',
 'writeoff_type_fraud_other',
 'writeoff_type_fraud_synthetic',
 'writeoff_type_null',
 'writeoff_type_other',
 'writeoff_type_repo',
 'writeoff_type_settlement']